In [1]:
from Bio.Blast import NCBIWWW
from Bio import SeqIO
from Bio.Blast import NCBIXML

help(NCBIWWW.qblast)

Help on function qblast in module Bio.Blast.NCBIWWW:

qblast(program, database, sequence, url_base='https://blast.ncbi.nlm.nih.gov/Blast.cgi', auto_format=None, composition_based_statistics=None, db_genetic_code=None, endpoints=None, entrez_query='(none)', expect=10.0, filter=None, gapcosts=None, genetic_code=None, hitlist_size=50, i_thresh=None, layout=None, lcase_mask=None, matrix_name=None, nucl_penalty=None, nucl_reward=None, other_advanced=None, perc_ident=None, phi_pattern=None, query_file=None, query_believe_defline=None, query_from=None, query_to=None, searchsp_eff=None, service=None, threshold=None, ungapped_alignment=None, word_size=None, short_query=None, alignments=500, alignment_view=None, descriptions=500, entrez_links_new_window=None, expect_low=None, expect_high=None, format_entrez_query=None, format_object=None, format_type='XML', ncbi_gi=None, results_file=None, show_overview=None, megablast=None, template_type=None, template_length=None, username='blast', password=No

In [2]:
sequence_data = open('blast_example.fasta').read()
sequence_data

'Example of a single sequence in FASTA/Pearson format: \n>sequence A ggtaagtcctctagtacaaacacccccaatattgtgatataattaaaattatattcatat\ntctgttgccagaaaaaacacttttaggctatattagagccatcttctttgaagcgttgtc \n\n>sequence B ggtaagtcctctagtacaaacacccccaatattgtgatataattaaaattatattca\ntattctgttgccagaaaaaacacttttaggctatattagagccatcttctttgaagcgttgtc'

In [3]:
result_handle = NCBIWWW.qblast('blastn', 'nt', sequence_data)
result_handle

In [4]:
seq_record = next(SeqIO.parse(open("blast_example.fasta"), "fasta"))
result_handle = NCBIWWW.qblast("blastn", "nt", seq_record.seq)
print(result_handle)

In [4]:
with open('results.xml', 'w') as f:
    blast_results = result_handle.read()
    f.write(blast_results)

In [7]:
E_VALUE_THRESH = 1e-15
for record in NCBIXML.parse(open("results.xml")):
    if record.alignments:
        print("\n")
        print("query: %s" % record.query[:100])
        for align in record.alignments:
            for hsp in align.hsps:
                if hsp.expect < E_VALUE_THRESH:
                    print("match: %s " % align.title[:100])



query: sequence A ggtaagtcctctagtacaaacacccccaatattgtgatataattaaaattatattcatat
match: gi|1853088208|gb|CP054431.1| Chlamydia trachomatis strain CH2_mutant_L2/434/Bu(i) plasmid unnamed 
match: gi|1853087206|gb|CP054433.1| Chlamydia trachomatis strain CH1_mutant_L2/434/Bu(i) plasmid unnamed 
match: gi|1853086301|gb|CP054429.1| Chlamydia trachomatis strain CH3_mutant_L2/434/Bu(i) plasmid unnamed 
match: gi|1853085398|gb|CP054427.1| Chlamydia trachomatis strain CH5_mutant_L2/434/Bu(i) plasmid unnamed 
match: gi|1834305694|gb|MT241513.1| Cloning vector pREF100, complete sequence 
match: gi|2577273933|gb|OR270825.1| Chlamydia trachomatis isolate 2CP_CRYP_G06 hypothetical protein gene, c 
match: gi|2577273931|gb|OR270824.1| Chlamydia trachomatis isolate 1CP_CRYP_F06 hypothetical protein gene, c 
match: gi|2317634782|gb|CP106986.1| Chlamydia trachomatis strain L2f plasmid p7Kb, complete sequence 
match: gi|575868421|gb|KF790910.1| Cloning vector pBOMB4-Tet-mCherry, complete sequence 
match: 

In [9]:
E_VALUE_THRESH = 1e-20
for record in NCBIXML.parse(open("results.xml")):
    if record.alignments:
        print("\n")
        print("query: %s" % record.query[:100])
        for align in record.alignments:
            for hsp in align.hsps:
                if hsp.expect < E_VALUE_THRESH:
                    print("****Alignment****")
                    print("sequence:", align.title)
                    print("length:", align.length)
                    print("e value:", hsp.expect)
                    print(hsp.query[0:75] + "...")
                    print(hsp.match[0:75] + "...")
                    print(hsp.sbjct[0:75] + "...")



query: sequence A ggtaagtcctctagtacaaacacccccaatattgtgatataattaaaattatattcatat


query: sequence B ggtaagtcctctagtacaaacacccccaatattgtgatataattaaaattatattca
****Alignment****
sequence: gi|1853088208|gb|CP054431.1| Chlamydia trachomatis strain CH2_mutant_L2/434/Bu(i) plasmid unnamed
length: 7676
e value: 9.94599e-22
TATTCTGTTGCCAGAAAAAACACTTTTAGGCTATATTAGAGCCATCTTCTTTGAAGCGTTGTC...
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||...
TATTCTGTTGCCAGAAAAAACACTTTTAGGCTATATTAGAGCCATCTTCTTTGAAGCGTTGTC...
****Alignment****
sequence: gi|1853087206|gb|CP054433.1| Chlamydia trachomatis strain CH1_mutant_L2/434/Bu(i) plasmid unnamed
length: 7676
e value: 9.94599e-22
TATTCTGTTGCCAGAAAAAACACTTTTAGGCTATATTAGAGCCATCTTCTTTGAAGCGTTGTC...
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||...
TATTCTGTTGCCAGAAAAAACACTTTTAGGCTATATTAGAGCCATCTTCTTTGAAGCGTTGTC...
****Alignment****
sequence: gi|1853086301|gb|CP054429.1| Chlamydia trachomatis strain CH3_mutant_L2/434/Bu(i) plasmid u

In [21]:
from Bio.Blast.Applications import NcbiblastnCommandline
from io import StringIO
from Bio.Blast import NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

# Run BLAST and parse the output as XML
cline = NcbiblastnCommandline(
    query="../virus_datasets/spike_covid/gene.fna",
    subject="../virus_datasets/sars.faa",
    outfmt=5,
    out="result_temp.xml",
)
print(cline)

blastn -out result_temp.xml -outfmt 5 -query ../virus_datasets/spike_covid/gene.fna -subject ../virus_datasets/sars.faa


In [19]:
cline()

ApplicationError: Non-zero return code 127 from 'blastn -out result_temp.xml -outfmt 5 -query ../virus_datasets/spike_covid/gene.fna -subject ../virus_datasets/sars.faa', message '/bin/sh: 1: blastn: not found'

In [ ]:
for record in NCBIXML.parse(open("result_temp.xml")):
    # Print some information on the result
    for alignment in record.alignments:
        for hsp in alignment.hsps:
            print("****Alignment****")
            print("sequence:", alignment.title)
            print("length:", alignment.length)
            print("e value:", hsp.expect)
            print(hsp.query[0:75] + "...")
            print(hsp.match[0:75] + "...")
            print(hsp.sbjct[0:75] + "...")